Benchmark Velocyto runtime
---

In this notebook, we benchmark the runtime of Velocytos's functions:

- estimate_transition_prob
- calculate_embedding_shift
- prepare_markov
- run_markov

# Preliminaries

## Dependency notebooks

1. [../preprocessing_notebooks/MK_2020-10-16_preprocess_data.ipynb](../preprocessing_notebooks/MK_2020-10-16_preprocess_data.ipynb)

## Import packages

In [1]:
# import standard packages
from pathlib import Path
from collections import defaultdict
from math import ceil
import sys
import pickle
import gc

import pandas as pd
import numpy as np

# import single-cell packages
import scanpy as sc
import velocyto as vcy
from anndata import AnnData

# import utilities
import utils.utilities as ul

No dynamo version found


## Print package versions for reproducibility

In [2]:
sc.logging.print_versions()
vcy.__version__

scanpy==1.5.1 anndata==0.7.5 umap==0.5.0 numpy==1.19.4 scipy==1.5.3 pandas==1.1.4 scikit-learn==0.23.2 statsmodels==0.12.1


'0.17.17'

## Set up paths

In [3]:
sys.path.insert(0, "../../..")  # this depends on the notebook depth and must be adapted per notebook

from paths import DATA_DIR

## Load the data

Load the preprocessed data containing velocities.

In [4]:
adata = sc.read(DATA_DIR / "morris_data" / "adata_preprocessed.h5ad")
del adata.layers['unspliced']
del adata.layers['ambiguous']
adata

AnnData object with n_obs × n_vars = 104679 × 1500
    obs: 'batch', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts', 'velocity_self_transition'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'fit_r2', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_steady', 'fit_steady_u', 'fit_steady_s', 'fit_variance', 'fit_alignment_scaling', 'velocity_genes'
    uns: 'neighbors', 'pca', 'recover_dynamics', 'velocity_graph', 'velocity_graph_neg', 'velocity_params'
    obsm: 'X_pca'
    varm: 'PCs', 'loss'
    layers: 'Ms', 'Mu', 'fit_t', 'fit_tau', 'fit_tau_', 'matrix', 'spliced', 'velocity', 'velocity_u'
    obsp: 'connectivities', 'distances'

### Load the subsets and splits

In [5]:
dfs = ul.get_split(DATA_DIR / "morris_data" / "splits")
list(dfs.keys())

[10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000]

## Define utility functions

In [6]:
def initialize(adata: AnnData) -> vcy.VelocytoLoom:
    print("Initializing Velocyto object")

    vlm = object.__new__(vcy.VelocytoLoom)
    vlm.ts = adata.obsm["X_pca"][:, :2]
    vlm.Sx_sz = adata.layers["Ms"].astype(np.float64).T  # needs to be float
    vlm.used_delta_t = 1.0
    vlm.delta_S = adata.layers["velocity"].T
    vlm.S = adata.layers["spliced"].T  # used to get n_neighs
    vlm.delta_S = np.ascontiguousarray(vlm.delta_S)

    return vlm


def run(vlm: vcy.VelocytoLoom, *, n_jobs: int = 32) -> dict:
    estimate_transition_prob = ul.timeit(vlm.estimate_transition_prob)
    calculate_embedding_shift = ul.timeit(vlm.calculate_embedding_shift)
    prepare_markov = ul.timeit(vlm.prepare_markov)
    run_markov = ul.timeit(vlm.run_markov)

    print("Calculating transition probabilities")
    _, etp_time = estimate_transition_prob(  # only works on 2D embedding
        hidim="Sx_sz", embed="ts", transform="sqrt", psc=1,
        n_neighbors=None, knn_random=True, n_jobs=n_jobs,
    )
    print("Embedding shape:", vlm.embedding.shape)
    print("Calculating embedding shift")
    _, ces_time = calculate_embedding_shift()

    print("Preparing Markov")
    _, pm_time = prepare_markov(sigma_D=1, sigma_W=0.5)
    print("Running Markov using")
    _, rm_time = run_markov()

    res  = {
        "estimate_transition_probability": etp_time,
        "calculate_embedding_shift": ces_time,
        "prepare_markov": pm_time,
        "run_markov": rm_time,
    }
    print("Result:", res)
    
    return res

In [7]:
def benchmark_velocyto(adata: AnnData) -> dict:
    res = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    for size, split in dfs.items():
        for col in split.columns:
            try:
                print(f"Subsetting data to `{size}`, split `{col}`.")
                ixs = split[col].values
                bdata = adata[ixs].copy()
                
                assert bdata.n_obs == size, (bdata.n_obs, size)
                
                res[size][col] = run(initialize(bdata), n_jobs=32)
                
                ul.save_results(res, DATA_DIR / "benchmarking" / "runtime_analysis" / "velocyto.pickle")
                
                del bdata
                gc.collect()
            except Exception as e:
                print(f"Unable to run `Velocyto` with size `{size}` on split `{col}`. Reason: `{e}`.")
                continue
                
    return res

# Run the benchmarks

In [ ]:
res_velo = benchmark_velocyto(adata)

Subsetting data to `10000`, split `0`.
Initializing Velocyto object
Calculating transition probabilities


Embedding shape: (10000, 2)
Calculating embedding shift
Preparing Markov
Running Markov using


## Save the results

In [ ]:
ul.save_results(res_velo, DATA_DIR / "benchmarking" / "runtime_analysis" / "velocyto.pickle")